In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd
df = pd.read_csv("spam.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

### Creating a separate column of spam

In [4]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['spam'], stratify=df['spam'])

In [6]:
X_train.head(4)

2026    Yes obviously, but you are the eggs-pert and t...
4357       Great. So should i send you my account number.
760     Call me when you/carlos is/are here, my phone'...
916     I could ask carlos if we could get more if any...
Name: Message, dtype: object

In [7]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [8]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [9]:
get_sentence_embedding([
    "Every one must attend the meet untill there is no college work",
    "Lemon is becoming expensive day by day"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7120298 , -0.2918304 , -0.4726486 , ..., -0.12249377,
        -0.60608447,  0.73499733],
       [-0.9077529 , -0.38561866, -0.37178776, ...,  0.07115798,
        -0.6896424 ,  0.9215837 ]], dtype=float32)>

In [21]:
e = get_sentence_embedding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[1]],[e[3]])     #embedded text after encoding have similarities

array([[0.975397]], dtype=float32)

In [12]:
cosine_similarity([e[0]],[e[1]])   

array([[0.87801373]], dtype=float32)

In [13]:
cosine_similarity([e[3]],[e[5]])   

array([[0.8125404]], dtype=float32)

### Functional model

In [15]:
#Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

#Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

#Use of inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [18]:
len(X_train)

4179

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 1331s 10s/step - loss: 0.3315 - accuracy: 0.8653
Epoch 2/5
131/131 [==============================] - 759s 6s/step - loss: 0.2409 - accuracy: 0.8942
Epoch 3/5
131/131 [==============================] - 827s 6s/step - loss: 0.2022 - accuracy: 0.9177
Epoch 4/5
131/131 [==============================] - 845s 6s/step - loss: 0.1747 - accuracy: 0.9320
Epoch 5/5
131/131 [==============================] - 839s 6s/step - loss: 0.1593 - accuracy: 0.9440


In [24]:
model.predict([
    'Hello! you have won the lucky draw in which you will be getting near about 50 crores',
    'hey harshendra are you there in the lab of machines',
    'If you want to win a mercedes scan the QR given below',
    'sir has asked to fill the form with your original ID',
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
])

array([[0.16077733],
       [0.02929887],
       [0.0563629 ],
       [0.08713368],
       [0.60146874],
       [0.6538893 ],
       [0.5135639 ],
       [0.06031483],
       [0.01992404]], dtype=float32)